# Lean 4 - Installation et Configuration

**Navigation** : ← (debut) | [Index](Lean-1-Setup.ipynb) | [Lean-2-Dependent-Types →](Lean-2-Dependent-Types.ipynb)

---


Ce notebook configure l'environnement pour la serie de notebooks Lean qui explore le proof assistant **Lean 4** pour la verification formelle de preuves mathematiques.

**IMPORTANT:** Ce notebook **Lean-1-Setup** configure uniquement l'environnement. Les exemples de preuves formelles se trouvent dans les notebooks suivants.

**Contenu de ce notebook:**
* Introduction a Lean 4
* Diagnostic de l'environnement
* Installation automatique des composants manquants
* Verification et premiers tests

**Serie complete Lean:**
1. **Lean-1-Setup** (ce notebook) - Installation et configuration
2. **Lean-2-Dependent-Types** - Systeme de types dependants
3. **Lean-3-Propositions-Proofs** - Logique propositionnelle et preuves
4. **Lean-4-Quantifiers** - Logique du premier ordre
5. **Lean-5-Tactics** - Construction de preuves par tactiques
6. **Lean-6-Mathlib-Essentials** - Bibliotheque mathematique Mathlib4
7. **Lean-7-LLM-Integration** - Integration avec les LLMs
8. **Lean-8-Agentic-Proving** - Agents autonomes pour preuves
9. **Lean-9-LeanDojo** - LeanDojo pour ML/LLM theorem proving

**Public vise:** Etudiants, chercheurs interesses par la verification formelle et les assistants de preuves.

*(Base sur Lean 4 - syntaxe moderne)*

---

## Plan de ce Notebook

1. [Introduction a Lean 4](#introduction)
2. [Diagnostic de l'environnement](#diagnostic)
3. [Installation automatique](#installation)
4. [Verification de l'installation](#verification)
5. [Premiers tests](#tests)
6. [Configuration VSCode (optionnel)](#vscode)
7. [Ressources et documentation](#ressources)

---

## 1. Introduction a Lean 4
<a id="introduction"></a>

**Lean 4** est un proof assistant (assistant de preuves) et un langage de programmation fonctionnel moderne. Il est base sur le **Calcul des Constructions** (Calculus of Constructions), une theorie des types dependants tres expressive.

### Cas d'usage

- **Verification formelle** de preuves mathematiques
- **Verification de programmes** (preuve de correction)
- **Formalisation de mathematiques** (Mathlib4 contient 4M+ lignes)
- **Enseignement de la logique** et des mathematiques formelles

### Lean vs autres proof assistants

| Aspect | Lean 4 | Coq | Isabelle | Z3 |
|--------|--------|-----|----------|----|
| Type | Proof assistant | Proof assistant | Proof assistant | SMT Solver |
| Fondation | Calcul des Constructions | CoC + Inductive | HOL | SAT/SMT |
| Tactiques | Oui | Oui | Oui | Automatique |
| Bibliotheque | Mathlib4 (4M+ lignes) | Standard Library | AFP | - |
| Performance | Tres rapide (Lean 4) | Moyenne | Rapide | Tres rapide |
| Courbe d'apprentissage | Moyenne-Elevee | Elevee | Moyenne | Faible |

### Avantages de Lean 4

- **Syntaxe moderne** et expressive
- **Performances excellentes** (compile)
- **Communaute active** (Mathlib4, Zulip chat)
- **Integration LLM** (LeanCopilot, AlphaProof)
- **Verification formelle** garantie (impossible de prouver du faux)

---

## 2. Diagnostic de l'environnement
<a id="diagnostic"></a>

Cette section analyse votre environnement et identifie les composants manquants.

### Composants requis

| Composant | Description | Requis |
|-----------|-------------|--------|
| **Python 3.8+** | Environnement d'execution | Oui |
| **elan** | Gestionnaire de versions Lean | Oui |
| **Lean 4** | Le proof assistant | Oui |
| **lean4_jupyter** | Kernel Jupyter pour Lean | Oui |
| **Kernel enregistre** | Lean dans Jupyter | Oui |

Executez la cellule suivante pour diagnostiquer votre environnement :

In [1]:
"""
Diagnostic complet de l'environnement Lean 4
Detecte les composants manquants et propose l'installation
"""

import subprocess
import shutil
import sys
import os
import platform
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, List, Tuple

@dataclass
class ComponentStatus:
    """Status d'un composant"""
    name: str
    installed: bool
    version: Optional[str] = None
    path: Optional[str] = None
    error: Optional[str] = None
    can_auto_install: bool = False

def check_python() -> ComponentStatus:
    """Verifie Python >= 3.8"""
    version = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
    ok = sys.version_info >= (3, 8)
    return ComponentStatus(
        name="Python",
        installed=ok,
        version=version,
        path=sys.executable,
        error=None if ok else "Python 3.8+ requis"
    )

def check_elan() -> ComponentStatus:
    """Verifie si elan est installe"""
    elan_path = shutil.which("elan")
    if elan_path:
        try:
            result = subprocess.run(["elan", "--version"], capture_output=True, text=True, timeout=10)
            version = result.stdout.strip().split('\n')[0] if result.returncode == 0 else "unknown"
            return ComponentStatus(name="elan", installed=True, version=version, path=elan_path)
        except Exception as e:
            return ComponentStatus(name="elan", installed=True, path=elan_path, error=str(e))
    return ComponentStatus(
        name="elan",
        installed=False,
        error="elan non trouve dans PATH",
        can_auto_install=True
    )

def check_lean() -> ComponentStatus:
    """Verifie si Lean 4 est installe"""
    lean_path = shutil.which("lean")
    if lean_path:
        try:
            result = subprocess.run(["lean", "--version"], capture_output=True, text=True, timeout=10)
            version = result.stdout.strip().split('\n')[0] if result.returncode == 0 else "unknown"
            return ComponentStatus(name="Lean 4", installed=True, version=version, path=lean_path)
        except Exception as e:
            return ComponentStatus(name="Lean 4", installed=True, path=lean_path, error=str(e))
    return ComponentStatus(
        name="Lean 4",
        installed=False,
        error="lean non trouve (installez via elan)",
        can_auto_install=True  # Via elan
    )

def check_lean4_jupyter() -> ComponentStatus:
    """Verifie si lean4_jupyter est installe"""
    try:
        import importlib.metadata
        version = importlib.metadata.version("lean4-jupyter")
        return ComponentStatus(name="lean4_jupyter", installed=True, version=version)
    except importlib.metadata.PackageNotFoundError:
        pass
    except Exception:
        pass
    
    # Fallback: essayer l'import
    try:
        import lean4_jupyter
        return ComponentStatus(name="lean4_jupyter", installed=True, version="unknown")
    except ImportError:
        return ComponentStatus(
            name="lean4_jupyter",
            installed=False,
            error="Module non installe",
            can_auto_install=True
        )

def check_jupyter_kernel() -> ComponentStatus:
    """Verifie si le kernel Lean est enregistre dans Jupyter"""
    try:
        result = subprocess.run(
            ["jupyter", "kernelspec", "list", "--json"],
            capture_output=True, text=True, timeout=10
        )
        if result.returncode == 0:
            import json
            kernels = json.loads(result.stdout)
            kernel_names = list(kernels.get("kernelspecs", {}).keys())
            lean_kernel = next((k for k in kernel_names if "lean" in k.lower()), None)
            if lean_kernel:
                return ComponentStatus(
                    name="Kernel Jupyter",
                    installed=True,
                    version=lean_kernel,
                    path=kernels["kernelspecs"][lean_kernel].get("resource_dir")
                )
        return ComponentStatus(
            name="Kernel Jupyter",
            installed=False,
            error="Kernel Lean non enregistre",
            can_auto_install=True
        )
    except Exception as e:
        return ComponentStatus(
            name="Kernel Jupyter",
            installed=False,
            error=str(e),
            can_auto_install=True
        )

def run_diagnostic() -> Tuple[List[ComponentStatus], bool]:
    """Execute le diagnostic complet"""
    print("=" * 60)
    print("    DIAGNOSTIC DE L'ENVIRONNEMENT LEAN 4")
    print("=" * 60)
    print(f"\nSysteme: {platform.system()} {platform.release()}")
    print(f"Architecture: {platform.machine()}")
    print()
    
    components = [
        check_python(),
        check_elan(),
        check_lean(),
        check_lean4_jupyter(),
        check_jupyter_kernel()
    ]
    
    print("-" * 60)
    print("Composant              Status       Version/Info")
    print("-" * 60)
    
    all_ok = True
    missing_installable = []
    
    for c in components:
        status = "[OK]" if c.installed else "[MANQUANT]"
        info = c.version or c.error or ""
        if c.path and c.installed:
            info = f"{info} ({c.path})" if info else c.path
        print(f"{c.name:<20}  {status:<12} {info[:35]}")
        
        if not c.installed:
            all_ok = False
            if c.can_auto_install:
                missing_installable.append(c.name)
    
    print("-" * 60)
    
    if all_ok:
        print("\n[OK] Tous les composants sont installes !")
        print("     Vous pouvez passer aux notebooks suivants.")
    else:
        print(f"\n[!] {len(missing_installable)} composant(s) manquant(s) peuvent etre installes automatiquement:")
        for name in missing_installable:
            print(f"    - {name}")
        print("\n    Executez la cellule 'Installation automatique' ci-dessous.")
    
    print("=" * 60)
    
    return components, all_ok

# Stocker le resultat pour utilisation ulterieure
DIAGNOSTIC_RESULTS, ALL_COMPONENTS_OK = run_diagnostic()

    DIAGNOSTIC DE L'ENVIRONNEMENT LEAN 4

Systeme: Linux 6.6.87.2-microsoft-standard-WSL2
Architecture: x86_64

------------------------------------------------------------
Composant              Status       Version/Info
------------------------------------------------------------
Python                [OK]         3.12.3 (/home/jesse/.python3-wsl-ve
elan                  [MANQUANT]   elan non trouve dans PATH
Lean 4                [MANQUANT]   lean non trouve (installez via elan
lean4_jupyter         [MANQUANT]   Module non installe
Kernel Jupyter        [MANQUANT]   [Errno 2] No such file or directory
------------------------------------------------------------

[!] 4 composant(s) manquant(s) peuvent etre installes automatiquement:
    - elan
    - Lean 4
    - lean4_jupyter
    - Kernel Jupyter

    Executez la cellule 'Installation automatique' ci-dessous.


---

## 3. Installation automatique
<a id="installation"></a>

Cette section installe automatiquement les composants manquants detectes ci-dessus.

### Processus d'installation

L'installation se fait dans l'ordre suivant :
1. **elan** - Telecharge et installe le gestionnaire de versions Lean
2. **Lean 4** - Installe la version stable via elan
3. **lean4_jupyter** - Installe le module Python via pip
4. **Kernel Jupyter** - Enregistre le kernel Lean dans Jupyter

**Note Windows:** L'installation de elan necessite PowerShell et peut demander une confirmation.

Executez la cellule suivante pour installer les composants manquants :

In [2]:
"""
Installation automatique des composants manquants
"""

import subprocess
import sys
import os
import platform
import shutil
import tempfile
import urllib.request
from pathlib import Path

class LeanInstaller:
    """Installateur automatique pour l'environnement Lean 4"""
    
    def __init__(self):
        self.system = platform.system()
        self.is_windows = self.system == "Windows"
        self.is_macos = self.system == "Darwin"
        self.is_linux = self.system == "Linux"
        self.results = []
        
    def log(self, message: str, level: str = "INFO"):
        """Log un message"""
        prefix = {"INFO": "[*]", "OK": "[+]", "ERROR": "[!]", "WARN": "[~]"}.get(level, "[*]")
        print(f"{prefix} {message}")
        self.results.append((level, message))
    
    def run_command(self, cmd: list, description: str, timeout: int = 300, shell: bool = False) -> bool:
        """Execute une commande et retourne True si succes"""
        self.log(f"{description}...")
        try:
            if shell and self.is_windows:
                # Pour PowerShell sur Windows
                result = subprocess.run(
                    cmd,
                    capture_output=True,
                    text=True,
                    timeout=timeout,
                    shell=True
                )
            else:
                result = subprocess.run(
                    cmd,
                    capture_output=True,
                    text=True,
                    timeout=timeout
                )
            
            if result.returncode == 0:
                self.log(f"{description} - OK", "OK")
                return True
            else:
                self.log(f"{description} - Erreur: {result.stderr[:200]}", "ERROR")
                return False
        except subprocess.TimeoutExpired:
            self.log(f"{description} - Timeout apres {timeout}s", "ERROR")
            return False
        except Exception as e:
            self.log(f"{description} - Exception: {e}", "ERROR")
            return False
    
    def install_elan(self) -> bool:
        """Installe elan (gestionnaire de versions Lean)"""
        # Verifier si deja installe
        if shutil.which("elan"):
            self.log("elan deja installe", "OK")
            return True
        
        self.log("Installation de elan...")
        
        if self.is_windows:
            return self._install_elan_windows()
        else:
            return self._install_elan_unix()
    
    def _install_elan_windows(self) -> bool:
        """Installation elan sur Windows via PowerShell"""
        try:
            # Telecharger le script d'installation
            script_url = "https://raw.githubusercontent.com/leanprover/elan/master/elan-init.ps1"
            
            with tempfile.NamedTemporaryFile(mode='w', suffix='.ps1', delete=False) as f:
                script_path = f.name
            
            self.log(f"Telechargement du script elan depuis {script_url}")
            urllib.request.urlretrieve(script_url, script_path)
            
            # Executer avec PowerShell (mode non-interactif)
            self.log("Execution du script d'installation (cela peut prendre quelques minutes)...")
            
            # Utiliser -Command pour permettre l'interpretation de $true par PowerShell
            # Echapper le chemin pour PowerShell
            escaped_path = script_path.replace("'", "''")
            ps_command = f"& '{escaped_path}' -NoPrompt:$true -DefaultToolchain none"
            
            cmd = [
                "powershell", "-ExecutionPolicy", "Bypass", "-Command", ps_command
            ]
            
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=600  # 10 minutes max
            )
            
            # Nettoyer
            os.unlink(script_path)
            
            if result.returncode == 0:
                # Mettre a jour le PATH pour cette session
                elan_bin = Path.home() / ".elan" / "bin"
                if elan_bin.exists():
                    os.environ["PATH"] = str(elan_bin) + os.pathsep + os.environ.get("PATH", "")
                    self.log("elan installe avec succes", "OK")
                    self.log(f"PATH mis a jour: {elan_bin}")
                    return True
            
            self.log(f"Erreur installation elan: {result.stderr[:300]}", "ERROR")
            self.log("Essayez l'installation manuelle - voir instructions ci-dessous", "WARN")
            return False
            
        except Exception as e:
            self.log(f"Exception lors de l'installation elan: {e}", "ERROR")
            return False
    
    def _install_elan_unix(self) -> bool:
        """Installation elan sur Linux/macOS"""
        try:
            # Utiliser curl pour telecharger et executer le script
            cmd = [
                "sh", "-c",
                "curl -sSf https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh | sh -s -- -y --default-toolchain none"
            ]
            
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=600
            )
            
            if result.returncode == 0:
                # Mettre a jour le PATH
                elan_bin = Path.home() / ".elan" / "bin"
                if elan_bin.exists():
                    os.environ["PATH"] = str(elan_bin) + os.pathsep + os.environ.get("PATH", "")
                    self.log("elan installe avec succes", "OK")
                    return True
            
            self.log(f"Erreur: {result.stderr[:200]}", "ERROR")
            return False
            
        except Exception as e:
            self.log(f"Exception: {e}", "ERROR")
            return False
    
    def install_lean4(self) -> bool:
        """Installe Lean 4 stable via elan"""
        # Verifier si deja installe
        if shutil.which("lean"):
            self.log("Lean 4 deja installe", "OK")
            return True
        
        # Verifier que elan est disponible
        elan_path = shutil.which("elan")
        if not elan_path:
            # Essayer le chemin par defaut
            elan_bin = Path.home() / ".elan" / "bin" / ("elan.exe" if self.is_windows else "elan")
            if elan_bin.exists():
                elan_path = str(elan_bin)
            else:
                self.log("elan non trouve - installez elan d'abord", "ERROR")
                return False
        
        self.log("Installation de Lean 4 (stable) via elan...")
        self.log("Cela peut prendre plusieurs minutes (telechargement ~200MB)...")
        
        return self.run_command(
            [elan_path, "default", "leanprover/lean4:stable"],
            "Installation Lean 4 stable",
            timeout=600
        )
    
    def install_lean4_jupyter(self) -> bool:
        """Installe le module lean4_jupyter via pip"""
        try:
            import lean4_jupyter
            self.log("lean4_jupyter deja installe", "OK")
            return True
        except ImportError:
            pass
        
        return self.run_command(
            [sys.executable, "-m", "pip", "install", "lean4-jupyter", "--quiet"],
            "Installation lean4_jupyter via pip",
            timeout=120
        )
    
    def register_jupyter_kernel(self) -> bool:
        """Enregistre le kernel Lean dans Jupyter"""
        # Verifier si deja enregistre
        try:
            result = subprocess.run(
                ["jupyter", "kernelspec", "list"],
                capture_output=True, text=True, timeout=10
            )
            if "lean" in result.stdout.lower():
                self.log("Kernel Lean deja enregistre", "OK")
                return True
        except Exception:
            pass
        
        return self.run_command(
            [sys.executable, "-m", "lean4_jupyter", "install"],
            "Enregistrement du kernel Jupyter",
            timeout=60
        )
    
    def install_all(self) -> bool:
        """Installe tous les composants manquants"""
        print("=" * 60)
        print("    INSTALLATION AUTOMATIQUE DES COMPOSANTS LEAN 4")
        print("=" * 60)
        print()
        
        steps = [
            ("1/4", "elan", self.install_elan),
            ("2/4", "Lean 4", self.install_lean4),
            ("3/4", "lean4_jupyter", self.install_lean4_jupyter),
            ("4/4", "Kernel Jupyter", self.register_jupyter_kernel),
        ]
        
        success = True
        for step_num, name, func in steps:
            print(f"\n--- Etape {step_num}: {name} ---")
            if not func():
                success = False
                self.log(f"Echec de l'installation de {name}", "ERROR")
                # Continuer quand meme pour les autres composants
        
        print("\n" + "=" * 60)
        if success:
            print("[OK] Installation terminee avec succes !")
            print("\nIMPORTANT: Redemarrez le kernel Jupyter pour prendre en compte les changements.")
            print("           Kernel > Restart Kernel (ou Ctrl+.)")
        else:
            print("[!] Installation incomplete - voir les erreurs ci-dessus")
            print("\nPour une installation manuelle, voir la section 'Installation manuelle' ci-dessous.")
        print("=" * 60)
        
        return success

# Verifier si l'installation est necessaire
try:
    if not ALL_COMPONENTS_OK:
        print("Des composants manquants ont ete detectes.")
        print("Lancement de l'installation automatique...\n")
        installer = LeanInstaller()
        installer.install_all()
    else:
        print("[OK] Tous les composants sont deja installes.")
        print("     Aucune action necessaire.")
except NameError:
    print("[!] Executez d'abord la cellule de diagnostic (section 2) avant l'installation.")

Des composants manquants ont ete detectes.
Lancement de l'installation automatique...

    INSTALLATION AUTOMATIQUE DES COMPOSANTS LEAN 4


--- Etape 1/4: elan ---
[*] Installation de elan...
[+] elan installe avec succes

--- Etape 2/4: Lean 4 ---
[+] Lean 4 deja installe

--- Etape 3/4: lean4_jupyter ---
[*] Installation lean4_jupyter via pip...
[+] Installation lean4_jupyter via pip - OK

--- Etape 4/4: Kernel Jupyter ---
[*] Enregistrement du kernel Jupyter...
[!] Enregistrement du kernel Jupyter - Erreur: Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/jesse/.python3-wsl-venv/lib/python3.12/site-pa
[!] Echec de l'installation de Kernel Jupyter

[!] Installation incomplete - voir les erreurs ci-dessus

Pour une installation manuelle, voir la section 'Installation manuelle' ci-dessous.


### Installation manuelle (si l'automatique echoue)

Si l'installation automatique echoue, suivez ces etapes manuellement :

#### Windows (PowerShell en administrateur)

```powershell
# 1. Installer elan
Invoke-WebRequest -Uri https://raw.githubusercontent.com/leanprover/elan/master/elan-init.ps1 -OutFile elan-init.ps1
.\elan-init.ps1

# 2. Redemarrer PowerShell puis installer Lean 4
elan default leanprover/lean4:stable

# 3. Installer lean4_jupyter
pip install lean4-jupyter
python -m lean4_jupyter install
```

#### Linux / macOS

```bash
# 1. Installer elan
curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh

# 2. Sourcer le profil et installer Lean 4
source ~/.elan/env
elan default leanprover/lean4:stable

# 3. Installer lean4_jupyter
pip install lean4-jupyter
python -m lean4_jupyter install
```

Apres installation, **redemarrez Jupyter** pour que les changements prennent effet.

---

## 4. Verification de l'installation
<a id="verification"></a>

Apres installation (ou redemarrage du kernel), executez cette cellule pour verifier que tout fonctionne :

In [3]:
"""
Verification finale de l'installation
"""

import subprocess
import shutil
import sys

print("=" * 60)
print("    VERIFICATION FINALE DE L'INSTALLATION")
print("=" * 60)
print()

checks = []

# 1. Python
python_ok = sys.version_info >= (3, 8)
checks.append(("Python 3.8+", python_ok, f"{sys.version_info.major}.{sys.version_info.minor}"))

# 2. elan
elan_path = shutil.which("elan")
elan_version = ""
if elan_path:
    try:
        result = subprocess.run(["elan", "--version"], capture_output=True, text=True, timeout=5)
        elan_version = result.stdout.strip().split()[1] if result.returncode == 0 else "?"
    except:
        elan_version = "?"
checks.append(("elan", bool(elan_path), elan_version))

# 3. lean
lean_path = shutil.which("lean")
lean_version = ""
if lean_path:
    try:
        result = subprocess.run(["lean", "--version"], capture_output=True, text=True, timeout=5)
        lean_version = result.stdout.strip().split('\n')[0] if result.returncode == 0 else "?"
    except:
        lean_version = "?"
checks.append(("Lean 4", bool(lean_path), lean_version))

# 4. lean4_jupyter module
try:
    import lean4_jupyter
    jupyter_mod_ok = True
    jupyter_mod_version = getattr(lean4_jupyter, "__version__", "installed")
except ImportError:
    jupyter_mod_ok = False
    jupyter_mod_version = ""
checks.append(("lean4_jupyter", jupyter_mod_ok, jupyter_mod_version))

# 5. Jupyter kernel
kernel_ok = False
kernel_name = ""
try:
    result = subprocess.run(["jupyter", "kernelspec", "list"], capture_output=True, text=True, timeout=10)
    if "lean" in result.stdout.lower():
        kernel_ok = True
        for line in result.stdout.split('\n'):
            if 'lean' in line.lower():
                kernel_name = line.strip().split()[0]
                break
except:
    pass
checks.append(("Kernel Jupyter", kernel_ok, kernel_name))

# Afficher les resultats
print(f"{'Composant':<20} {'Status':<12} {'Version/Info'}")
print("-" * 60)

all_ok = True
for name, ok, version in checks:
    status = "[OK]" if ok else "[MANQUANT]"
    print(f"{name:<20} {status:<12} {version}")
    if not ok:
        all_ok = False

print("-" * 60)

if all_ok:
    print("\n" + "*" * 60)
    print("*  INSTALLATION COMPLETE ET FONCTIONNELLE !              *")
    print("*                                                        *")
    print("*  Vous pouvez maintenant :                              *")
    print("*  1. Creer un nouveau notebook avec le kernel 'Lean 4'  *")
    print("*  2. Continuer avec Lean-2-Dependent-Types.ipynb        *")
    print("*" * 60)
else:
    print("\n[!] Installation incomplete.")
    print("    Executez la cellule d'installation automatique ci-dessus,")
    print("    puis redemarrez le kernel et re-executez cette verification.")

    VERIFICATION FINALE DE L'INSTALLATION

Composant            Status       Version/Info
------------------------------------------------------------
Python 3.8+          [OK]         3.12
elan                 [OK]         4.1.2
Lean 4               [OK]         Lean (version 4.11.0, x86_64-unknown-linux-gnu, commit ec3042d94bd1, Release)
lean4_jupyter        [OK]         0.0.2
Kernel Jupyter       [MANQUANT]   
------------------------------------------------------------

[!] Installation incomplete.
    Executez la cellule d'installation automatique ci-dessus,
    puis redemarrez le kernel et re-executez cette verification.


---

## 5. Premiers tests avec LeanRunner
<a id="tests"></a>

Le kernel `lean4_jupyter` a des problemes de compatibilite sur Windows (`signal.SIGPIPE` non disponible).

**Solution alternative** : Utiliser `LeanRunner`, une classe Python qui execute Lean via subprocess.
Cette approche fonctionne sur tous les systemes (Windows, Linux, macOS).

### Avantages de LeanRunner

| Aspect | lean4_jupyter | LeanRunner |
|--------|---------------|------------|
| Windows | Bug SIGPIPE | OK |
| Linux/macOS | OK | OK |
| Kernel separe | Oui | Non (Python) |
| Integration Python | Non | Oui |

Executez la cellule suivante pour tester Lean directement depuis Python :

In [4]:
"""
Test de Lean 4 via LeanRunner (fonctionne sur Windows)
"""

# Importer LeanRunner depuis le fichier local
import sys
from pathlib import Path

# Ajouter le repertoire courant au path si necessaire
notebook_dir = Path(__file__).parent if "__file__" in dir() else Path(".").resolve()
if str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

from lean_runner import LeanRunner, run_lean

# Creer une instance de LeanRunner
runner = LeanRunner()

# Verifier l'installation
print("=" * 60)
print("    TEST DE LEAN 4 VIA LEANRUNNER")
print("=" * 60)

info = runner.check_installation()
print(f"\nInstallation Lean:")
print(f"  Installe: {info['installed']}")
print(f"  Chemin: {info['path']}")
print(f"  Version: {info['version']}")

if info['installed']:
    # Test 1: Hello World
    print("\n--- Test 1: Hello World ---")
    result = runner.run('#eval "Hello from Lean 4!"')
    print(f"Output: {result.output}")
    print(f"Success: {result.success}")
    
    # Test 2: Arithmetique
    print("\n--- Test 2: Arithmetique ---")
    print(f"2 + 2 = {runner.eval('2 + 2')}")
    print(f"5 * 7 = {runner.eval('5 * 7')}")
    print(f"List.range 5 = {runner.eval('List.range 5')}")
    
    # Test 3: Types
    print("\n--- Test 3: Verification de types ---")
    print(runner.check("Nat"))
    print(runner.check("Nat -> Nat"))
    print(runner.check("List Nat"))
    
    # Test 4: Theoreme
    print("\n--- Test 4: Preuve de theoreme ---")
    result = runner.prove(
        "(a b : Nat) : a + b = b + a",
        "exact Nat.add_comm a b"
    )
    print(f"Preuve valide: {result.success}")
    print(f"Output: {result.output}")
    
    # Test 5: Mode interactif
    print("\n--- Test 5: Mode interactif ---")
    runner.run_interactive("""
-- Definition de fonction
def double (n : Nat) : Nat := n * 2

#eval double 21
#check double
""")
    
    print("\n" + "=" * 60)
    print("[OK] Tous les tests Lean ont reussi !")
    print("=" * 60)
else:
    print("\n[!] Lean n'est pas installe.")
    print("    Executez la cellule d'installation automatique ci-dessus.")

# Nettoyer
runner.cleanup()

ModuleNotFoundError: No module named 'lean_runner'

### Fix Windows : Patch du kernel lean4

Sur Windows, le kernel `lean4_jupyter` echoue avec l'erreur `signal.SIGPIPE not found`.

**Solution 1 (recommandee)** : Patcher le kernel Windows natif

Executez la cellule suivante pour patcher automatiquement le kernel `lean4`. Ce patch rend le kernel compatible Windows en ignorant `signal.SIGPIPE` qui n'existe pas sur cette plateforme.

**Solution 2 (alternative)** : Kernel WSL

Si vous preferez utiliser WSL ou si le patch echoue, la cellule suivante configure un kernel qui execute lean4_jupyter depuis WSL Ubuntu.

In [ ]:
"""
Installation automatique de REPL pour lean4_jupyter

REPL est le serveur REPL pour Lean 4, nécessaire pour le kernel Jupyter.
"""

import subprocess
import shutil
import platform
import tempfile
from pathlib import Path

def install_repl():
    """Installe le serveur REPL pour Lean 4"""
    
    # Vérifier si repl est déjà installé
    if shutil.which("repl") or shutil.which("repl.exe"):
        print("[OK] repl est déjà installé")
        return True
    
    print("=" * 60)
    print("    INSTALLATION DE REPL")
    print("=" * 60)
    print()
    
    # Chemins
    is_windows = platform.system() == "Windows"
    home = Path.home()
    elan_bin = home / ".elan" / "bin"
    repl_dir = home / "repl"
    
    print(f"[*] Vérification de lake...")
    if not shutil.which("lake"):
        print("[!] Lake n'est pas installé - installez Lean 4 d'abord")
        return False
    
    # Cloner le dépôt repl si nécessaire
    if not repl_dir.exists():
        print(f"[*] Clonage de repl depuis GitHub...")
        result = subprocess.run(
            ["git", "clone", "https://github.com/leanprover-community/repl", str(repl_dir)],
            capture_output=True, text=True, timeout=60
        )
        if result.returncode != 0:
            print(f"[!] Erreur lors du clonage: {result.stderr[:200]}")
            return False
        print("[+] Clonage réussi")
    else:
        print(f"[OK] Dépôt repl existe: {repl_dir}")
    
    # Construire repl
    print(f"[*] Construction de repl (cela peut prendre 1-2 minutes)...")
    result = subprocess.run(
        ["lake", "build"],
        cwd=str(repl_dir),
        capture_output=True, text=True, timeout=300
    )
    if result.returncode != 0:
        print(f"[!] Erreur lors de la construction: {result.stderr[:200]}")
        return False
    print("[+] Construction réussie")
    
    # Copier l'exécutable dans le PATH
    repl_exe = repl_dir / ".lake" / "build" / "bin" / ("repl.exe" if is_windows else "repl")
    if not repl_exe.exists():
        print(f"[!] Exécutable non trouvé: {repl_exe}")
        return False
    
    # Créer le répertoire elan/bin si nécessaire
    elan_bin.mkdir(parents=True, exist_ok=True)
    
    # Copier repl
    dest = elan_bin / repl_exe.name
    shutil.copy(repl_exe, dest)
    print(f"[+] repl copié vers: {dest}")
    
    # Vérifier l'installation
    if shutil.which("repl") or shutil.which("repl.exe"):
        print("\n" + "=" * 60)
        print("[OK] REPL installé avec succès !")
        print("=" * 60)
        return True
    else:
        print("\n" + "=" * 60)
        print("[~] REPL installé mais pas dans le PATH")
        print(f"    Ajoutez {elan_bin} au PATH et redémarrez")
        print("=" * 60)
        return True

# Installer repl
try:
    install_repl()
except Exception as e:
    print(f"\n[!] Erreur: {e}")
    print("\nInstallation manuelle:")
    print("  cd ~")
    print("  git clone https://github.com/leanprover-community/repl")
    print("  cd repl")
    print("  lake build")
    print("  cp .lake/build/bin/repl* ~/.elan/bin/")

[OK] repl est déjà installé


### Installation de REPL

Le kernel `lean4_jupyter` nécessite l'outil `repl` (REPL server pour Lean 4).

Exécutez la cellule suivante pour installer `repl` automatiquement :

In [ ]:
"""
Patch du kernel lean4_jupyter pour Windows (fix SIGPIPE)

Sur Windows, signal.SIGPIPE n'existe pas. Cette cellule patche
automatiquement le fichier kernel.py pour ignorer cette erreur.
"""

import sys
import platform
from pathlib import Path

def patch_lean4_jupyter_windows():
    """Patche lean4_jupyter pour Windows (SIGPIPE fix)"""
    
    if platform.system() != "Windows":
        print("[OK] Vous n'etes pas sur Windows - patch non necessaire")
        return True
    
    try:
        import lean4_jupyter
        kernel_file = Path(lean4_jupyter.__file__).parent / "kernel.py"
        
        if not kernel_file.exists():
            print(f"[!] Fichier kernel.py introuvable: {kernel_file}")
            return False
        
        # Lire le fichier
        with open(kernel_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Verifier si deja patche
        if "hasattr(signal, 'SIGPIPE')" in content:
            print(f"[OK] kernel.py deja patche: {kernel_file}")
            return True
        
        # Verifier si le bug existe
        if "signal.SIGPIPE" not in content:
            print(f"[OK] kernel.py ne contient pas de reference a SIGPIPE")
            return True
        
        # Appliquer le patch
        print(f"[*] Patch de {kernel_file}...")
        
        # Ligne 51: old_sigpipe_handler = signal.signal(signal.SIGPIPE, signal.SIG_DFL)
        old_line = "        old_sigpipe_handler = signal.signal(signal.SIGPIPE, signal.SIG_DFL)"
        new_line = "        # SIGPIPE doesn't exist on Windows, only set it if available\n        old_sigpipe_handler = signal.signal(signal.SIGPIPE, signal.SIG_DFL) if hasattr(signal, 'SIGPIPE') else None"
        
        content = content.replace(old_line, new_line)
        
        # Ligne 56: signal.signal(signal.SIGPIPE, old_sigpipe_handler)
        old_line2 = "            signal.signal(signal.SIGPIPE, old_sigpipe_handler)"
        new_line2 = "            if old_sigpipe_handler is not None:\n                signal.signal(signal.SIGPIPE, old_sigpipe_handler)"
        
        content = content.replace(old_line2, new_line2)
        
        # Ecrire le fichier patche
        with open(kernel_file, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"[+] Patch applique avec succes !")
        print(f"    Le kernel 'lean4' (Windows natif) est maintenant fonctionnel.")
        return True
        
    except ImportError:
        print("[!] lean4_jupyter non installe - installez-le d'abord")
        return False
    except Exception as e:
        print(f"[!] Erreur lors du patch: {e}")
        return False

# Appliquer le patch
print("=" * 60)
print("    PATCH DU KERNEL LEAN4 POUR WINDOWS")
print("=" * 60)
print()

if patch_lean4_jupyter_windows():
    print("\n" + "=" * 60)
    print("[OK] Kernel lean4 pret a l'emploi !")
    print("\n     Vous pouvez maintenant utiliser le kernel 'lean4'")
    print("     directement dans VSCode sans WSL.")
    print("\n     Rechargez VSCode: Ctrl+Shift+P > 'Reload Window'")
    print("=" * 60)
else:
    print("\n" + "=" * 60)
    print("[!] Patch echoue - utilisez le kernel WSL a la place")
    print("    Voir la cellule suivante pour WSL.")
    print("=" * 60)

    PATCH DU KERNEL LEAN4 POUR WINDOWS

[OK] kernel.py deja patche: c:\ProgramData\miniconda3\Lib\site-packages\lean4_jupyter\kernel.py

[OK] Kernel lean4 pret a l'emploi !

     Vous pouvez maintenant utiliser le kernel 'lean4'
     directement dans VSCode sans WSL.

     Rechargez VSCode: Ctrl+Shift+P > 'Reload Window'


In [ ]:
"""
Detection et configuration du kernel Lean4 via WSL (Windows uniquement)

Sur Windows, le kernel lean4_jupyter natif peut avoir des problemes avec les chemins.
Ce script configure automatiquement un kernel WSL robuste pour Jupyter/VSCode.

Fonctionnalites du wrapper robuste:
- Conversion de chemins Windows vers WSL (y compris chemins "mangles")
- PATH propre sans pollution Windows
- Lancement direct via IPKernelApp (plus fiable que -m)
- Logging pour debugging (~/.lean4-wrapper.log)
"""

import subprocess
import platform
import json
import os
from pathlib import Path

class WSLKernelManager:
    """Gestionnaire du kernel Lean4 via WSL avec wrapper robuste"""
    
    def __init__(self):
        self.is_windows = platform.system() == "Windows"
        self.kernel_name = "lean4-wsl"
        self.kernel_path = Path(os.environ.get("APPDATA", "")) / "jupyter" / "kernels" / self.kernel_name
        self.wrapper_path_wsl = "/home/jesse/.lean4-kernel-wrapper.py"
        self.venv_python = "/home/jesse/.lean4-venv/bin/python3"
    
    def _safe_decode(self, data: bytes) -> str:
        """Decode bytes safely, handling non-UTF8 characters from WSL"""
        return data.decode('utf-8', errors='replace')
    
    def _get_robust_wrapper_content(self) -> str:
        """Retourne le contenu du wrapper robuste avec toutes les corrections"""
        return r'''#!/usr/bin/env python3
"""
Lean 4 Jupyter Kernel Wrapper for WSL (Robust Version)

This script converts Windows paths to WSL paths and launches the lean4_jupyter kernel.
It handles various path formats that VSCode/Jupyter might pass:
- Standard Windows paths: C:\Users\...
- Tilde shorthand: ~\AppData\... or ~/AppData/...
- Mangled paths (backslashes eaten): c:UsersjsboiAppData...
"""
import sys
import subprocess
import os
import re


def convert_windows_path(path):
    """Convert Windows path to WSL path, handling various formats."""
    if not path:
        return path

    # Already a Unix path
    if path.startswith('/'):
        return path

    # Handle ~ shorthand (Windows-style: ~\AppData or ~/AppData)
    if path.startswith('~\\') or path.startswith('~/'):
        users_dir = '/mnt/c/Users'
        if os.path.exists(users_dir):
            for user in os.listdir(users_dir):
                user_lower = user.lower()
                if user_lower not in ['public', 'default', 'default user', 'all users']:
                    user_path = os.path.join(users_dir, user)
                    if os.path.isdir(user_path) and os.path.exists(os.path.join(user_path, 'AppData')):
                        win_home = f'/mnt/c/Users/{user}'
                        rest = path[1:].lstrip('\\').lstrip('/')
                        rest = rest.replace('\\', '/')
                        return f'{win_home}/{rest}'
        rest = path[1:].lstrip('\\').lstrip('/').replace('\\', '/')
        return f'/mnt/c/Users/jsboi/{rest}'

    # Check for mangled path FIRST (backslashes eaten): c:UsersjsboiAppData...
    if len(path) >= 2 and path[1] == ':':
        if path[2:].startswith('Users') and '\\' not in path and '/' not in path[2:]:
            match = re.match(r'([a-zA-Z]):Users([a-z0-9_]+)AppDataRoaming(.+)', path, re.IGNORECASE)
            if match:
                drive = match.group(1).lower()
                user = match.group(2)
                rest = match.group(3)
                rest = rest.replace('jupyter', '/jupyter').replace('runtime', '/runtime')
                rest = rest.replace('kernel-', '/kernel-')
                rest = rest.lstrip('/')
                return f'/mnt/{drive}/Users/{user}/AppData/Roaming/{rest}'

        # Standard Windows path - use wslpath
        try:
            result = subprocess.run(['wslpath', '-a', path], capture_output=True, text=True, timeout=5)
            if result.returncode == 0:
                return result.stdout.strip()
        except:
            pass
        # Fallback manual conversion
        drive = path[0].lower()
        rest = path[2:].replace('\\', '/').lstrip('/')
        return f'/mnt/{drive}/{rest}'

    return path


def main():
    log_file = os.path.expanduser('~/.lean4-wrapper.log')

    def log(msg):
        try:
            with open(log_file, 'a') as f:
                f.write(f"{msg}\n")
        except:
            pass

    log(f"=== Wrapper started ===")
    log(f"sys.argv: {sys.argv}")

    # Process arguments
    args = sys.argv[1:]
    for i, arg in enumerate(args):
        if arg == '-f' and i + 1 < len(args):
            original = args[i + 1]
            converted = convert_windows_path(args[i + 1])
            args[i + 1] = converted
            log(f"Original: {original}")
            log(f"Converted: {converted}")

            if os.path.exists(converted):
                log(f"Connection file EXISTS")
            else:
                log(f"Connection file MISSING - waiting...")
                import time
                for _ in range(20):
                    time.sleep(0.1)
                    if os.path.exists(converted):
                        log(f"Connection file appeared")
                        break
                else:
                    log(f"Connection file still missing after wait")
            break

    # Set up environment with CLEAN PATH (not inheriting polluted Windows PATH)
    os.environ['PATH'] = '/home/jesse/.elan/bin:/home/jesse/.lean4-venv/bin:/usr/local/bin:/usr/bin:/bin'
    os.chdir(os.path.expanduser('~'))
    log(f"PATH set (clean): {os.environ['PATH']}")
    log(f"cwd: {os.getcwd()}")
    log(f"About to launch kernel with args: {args}")

    # Launch the kernel directly using IPKernelApp.launch_instance
    # NOTE: python -m lean4_jupyter.kernel exits immediately for unknown reasons
    log(f"Launching kernel directly via IPKernelApp.launch_instance")

    try:
        sys.argv = ['lean4_jupyter'] + args
        log(f"sys.argv set to: {sys.argv}")

        from ipykernel.kernelapp import IPKernelApp
        from lean4_jupyter.kernel import Lean4Kernel

        log(f"Starting kernel...")
        IPKernelApp.launch_instance(kernel_class=Lean4Kernel)
        log(f"Kernel exited normally")
    except SystemExit as e:
        log(f"SystemExit: {e}")
        sys.exit(e.code if e.code is not None else 0)
    except Exception as e:
        log(f"Exception launching kernel: {e}")
        import traceback
        log(traceback.format_exc())
        sys.exit(1)


if __name__ == '__main__':
    main()
'''
    
    def _create_wrapper_script(self) -> bool:
        """Deploie le script wrapper robuste dans WSL"""
        wrapper_content = self._get_robust_wrapper_content()
        
        try:
            # Ecrire le wrapper via heredoc
            result = subprocess.run(
                ["wsl.exe", "-d", "Ubuntu", "--", "bash", "-c",
                 f"cat > {self.wrapper_path_wsl} << 'WRAPPER_EOF'\n{wrapper_content}\nWRAPPER_EOF\nchmod +x {self.wrapper_path_wsl}"],
                capture_output=True, timeout=10
            )
            if result.returncode == 0:
                print(f"[+] Wrapper robuste deploye: {self.wrapper_path_wsl}")
                return True
            print(f"[!] Erreur creation wrapper: {self._safe_decode(result.stderr)[:200]}")
            return False
        except Exception as e:
            print(f"[!] Exception: {e}")
            return False
        
    def check_wsl_available(self) -> dict:
        """Verifie si WSL est disponible"""
        if not self.is_windows:
            return {"available": False, "reason": "Non Windows - WSL non necessaire"}
        
        try:
            result = subprocess.run(
                ["wsl.exe", "--status"],
                capture_output=True, timeout=10
            )
            if result.returncode == 0:
                return {"available": True, "reason": "WSL disponible"}
            return {"available": False, "reason": "WSL non configure"}
        except FileNotFoundError:
            return {"available": False, "reason": "WSL non installe"}
        except Exception as e:
            return {"available": False, "reason": str(e)}
    
    def check_wsl_lean_ready(self) -> dict:
        """Verifie si Lean est configure dans WSL"""
        try:
            result = subprocess.run(
                ["wsl.exe", "-d", "Ubuntu", "--", "bash", "-c",
                 "source ~/.elan/env 2>/dev/null && lean --version && which python3"],
                capture_output=True, timeout=30
            )
            stdout = self._safe_decode(result.stdout)
            
            if result.returncode == 0 and "Lean" in stdout:
                lines = stdout.strip().split('\n')
                lean_version = lines[0] if lines else "unknown"
                return {"ready": True, "lean_version": lean_version}
            return {"ready": False, "reason": "Lean non installe dans WSL"}
        except Exception as e:
            return {"ready": False, "reason": str(e)}
    
    def check_wsl_kernel_registered(self) -> dict:
        """Verifie si le kernel WSL est enregistre"""
        try:
            result = subprocess.run(
                ["jupyter", "kernelspec", "list", "--json"],
                capture_output=True, timeout=10
            )
            stdout = self._safe_decode(result.stdout)
            if result.returncode == 0:
                kernels = json.loads(stdout)
                if self.kernel_name in kernels.get("kernelspecs", {}):
                    return {
                        "registered": True,
                        "path": kernels["kernelspecs"][self.kernel_name].get("resource_dir")
                    }
            return {"registered": False}
        except Exception as e:
            return {"registered": False, "error": str(e)}
    
    def install_wsl_kernel(self, force: bool = False) -> bool:
        """Installe le kernel Lean4-WSL avec wrapper Python robuste."""
        print("[*] Installation du kernel Lean4-WSL...")
        
        # Deployer le wrapper robuste dans WSL
        if not self._create_wrapper_script():
            print("[!] Echec deploiement du wrapper")
            return False
        
        # Creer le dossier du kernel
        self.kernel_path.mkdir(parents=True, exist_ok=True)
        
        # Creer kernel.json qui utilise le venv Python et le wrapper
        kernel_json = {
            "argv": [
                "wsl.exe", "-d", "Ubuntu", "--",
                self.venv_python, self.wrapper_path_wsl,
                "-f", "{connection_file}"
            ],
            "display_name": "Lean 4 (WSL)",
            "language": "lean4"
        }
        
        kernel_file = self.kernel_path / "kernel.json"
        with open(kernel_file, 'w') as f:
            json.dump(kernel_json, f, indent=2)
        
        print(f"[+] Kernel cree: {kernel_file}")
        return True
    
    def run_diagnostic(self):
        """Execute le diagnostic complet WSL et installe/met a jour le kernel"""
        print("=" * 60)
        print("    DIAGNOSTIC ET CONFIGURATION KERNEL LEAN4 VIA WSL")
        print("=" * 60)
        
        if not self.is_windows:
            print("\n[OK] Vous n'etes pas sur Windows.")
            print("     Utilisez le kernel lean4 standard.")
            return
        
        # 1. Verifier WSL
        print("\n[1/4] Verification de WSL...")
        wsl_status = self.check_wsl_available()
        if wsl_status["available"]:
            print(f"      [OK] {wsl_status['reason']}")
        else:
            print(f"      [!] {wsl_status['reason']}")
            print("\n      Pour installer WSL:")
            print("      wsl --install -d Ubuntu")
            return
        
        # 2. Verifier Lean dans WSL
        print("\n[2/4] Verification de Lean dans WSL...")
        lean_status = self.check_wsl_lean_ready()
        if lean_status.get("ready"):
            print(f"      [OK] {lean_status['lean_version']}")
        else:
            print(f"      [!] {lean_status.get('reason', 'Non configure')}")
            print("\n      Pour installer Lean dans WSL, executez dans WSL Ubuntu:")
            print("      curl -sSf https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh | sh")
            print("      source ~/.elan/env")
            print("      elan default leanprover/lean4:v4.11.0")
            print("      python3 -m venv ~/.lean4-venv")
            print("      source ~/.lean4-venv/bin/activate")
            print("      pip install lean4-jupyter ipykernel")
            return
        
        # 3. Installer/mettre a jour le kernel
        print("\n[3/4] Installation du kernel Jupyter...")
        kernel_status = self.check_wsl_kernel_registered()
        if kernel_status.get("registered"):
            print(f"      [OK] Kernel enregistre: {kernel_status['path']}")
            print("      [*] Mise a jour du wrapper robuste...")
        else:
            print("      [!] Kernel non enregistre")
            print("      [*] Installation automatique...")
        self.install_wsl_kernel(force=True)
        
        # 4. Tester le kernel
        print("\n[4/4] Test du kernel WSL...")
        test_ok = self._test_kernel()
        
        print("\n" + "=" * 60)
        if test_ok:
            print("[OK] Kernel Lean4-WSL pret a l'emploi !")
            print("\n     Dans VSCode:")
            print("     1. Rechargez la fenetre: Ctrl+Shift+P > 'Reload Window'")
            print("     2. Ouvrez un notebook .ipynb")
            print("     3. Cliquez sur 'Select Kernel' en haut a droite")
            print("     4. Choisissez 'Lean 4 (WSL)'")
            print("\n     Logs de debug: wsl cat ~/.lean4-wrapper.log")
        else:
            print("[~] Kernel installe mais test echoue.")
            print("    Redemarrez VSCode et reessayez.")
            print("    Verifiez les logs: wsl cat ~/.lean4-wrapper.log")
        print("=" * 60)
    
    def _test_kernel(self) -> bool:
        """Test rapide du kernel WSL"""
        try:
            result = subprocess.run(
                ["wsl.exe", "-d", "Ubuntu", "--", "bash", "-c",
                 f"source ~/.lean4-venv/bin/activate 2>/dev/null && "
                 f"source ~/.elan/env 2>/dev/null && "
                 f"python3 -c 'import lean4_jupyter; print(\"OK\")'"],
                capture_output=True, timeout=15
            )
            stdout = self._safe_decode(result.stdout)
            if "OK" in stdout:
                print("      [OK] lean4_jupyter fonctionne dans WSL")
                return True
            print(f"      [!] Erreur: {self._safe_decode(result.stderr)[:100]}")
            return False
        except Exception as e:
            print(f"      [!] Exception: {e}")
            return False

# Executer le diagnostic et configurer le kernel
wsl_manager = WSLKernelManager()
wsl_manager.run_diagnostic()

### Configuration du kernel Python WSL (pour notebooks 7-8)

Les notebooks **Lean-7-LLM-Integration** et **Lean-8-Agentic-Proving** utilisent un kernel Python qui s'exécute dans WSL pour avoir accès à l'environnement Lean.

Exécutez la cellule suivante pour configurer automatiquement ce kernel :

In [ ]:
"""
Configuration automatique du kernel Python 3 (WSL)
Installe python-dotenv, openai, anthropic dans le venv WSL
"""

import subprocess
import platform

if platform.system() == "Windows":
    print("=== Configuration Python 3 (WSL) ===")
    
    # Exécuter le script de setup
    script_path = "/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/scripts/setup_wsl_python.sh"
    
    result = subprocess.run(
        ["wsl", "-d", "Ubuntu", "bash", script_path],
        capture_output=True,
        text=True,
        timeout=120
    )
    
    if result.returncode == 0:
        print(result.stdout)
        print("\n[OK] Kernel Python 3 (WSL) configuré avec succès")
    else:
        print(f"[ERREUR] {result.stderr}")
        print("\nInstallation manuelle:")
        print("  wsl -d Ubuntu")
        print("  bash /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/scripts/setup_wsl_python.sh")
else:
    print("Non-Windows - configuration non nécessaire")

---

## 6. Configuration VSCode (optionnel)
<a id="vscode"></a>

Pour une meilleure experience de developpement avec Lean 4, VSCode avec l'extension Lean 4 est recommande.

### Installation de l'extension

1. Ouvrez VSCode
2. Allez dans Extensions (Ctrl+Shift+X)
3. Recherchez "Lean 4"
4. Installez l'extension officielle "lean4"

### Fonctionnalites de l'extension

- **Infoview** : Affichage en temps reel des buts et hypotheses
- **Go to Definition** : Navigation dans le code
- **Autocompletion** : Suggestions intelligentes
- **Diagnostics** : Erreurs et warnings en direct
- **Hover** : Information sur les types au survol

### Creation d'un projet Lean

```bash
# Creer un nouveau projet
lake new my_project
cd my_project

# Ouvrir dans VSCode
code .
```

L'extension Lean 4 se chargera automatiquement et vous pourrez commencer a ecrire des preuves.

---

## 7. Ressources et documentation
<a id="ressources"></a>

### Documentation officielle

- [Theorem Proving in Lean 4](https://leanprover.github.io/theorem_proving_in_lean4/) - Tutoriel complet
- [Lean 4 Documentation](https://leanprover.github.io/lean4/doc/) - Documentation technique
- [Mathematics in Lean](https://leanprover-community.github.io/mathematics_in_lean/) - Mathematiques avec Lean
- [Functional Programming in Lean](https://lean-lang.org/functional_programming_in_lean/) - Programmation fonctionnelle

### Mathlib4

- [Mathlib4 Documentation](https://leanprover-community.github.io/mathlib4_docs/) - API complete
- [Mathlib4 GitHub](https://github.com/leanprover-community/mathlib4) - Code source
- [Loogle](https://loogle.lean-lang.org/) - Recherche de theoremes

### Communaute

- [Zulip Chat](https://leanprover.zulipchat.com/) - Forum communautaire tres actif
- [GitHub Discussions](https://github.com/leanprover/lean4/discussions) - Discussions techniques

### Outils et integrations

- [lean4_jupyter GitHub](https://github.com/utensil/lean4_jupyter) - Ce kernel Jupyter
- [LeanCopilot](https://github.com/lean-dojo/LeanCopilot) - Assistance LLM (notebook 7)

---

## Notebooks suivants

Apres avoir complete ce setup, continuez avec :

**Partie 1 : Fondations**
- **[Lean-2-Dependent-Types.ipynb](Lean-2-Dependent-Types.ipynb)** - Systeme de types dependants
- **[Lean-3-Propositions-Proofs.ipynb](Lean-3-Propositions-Proofs.ipynb)** - Logique et preuves
- **[Lean-4-Quantifiers.ipynb](Lean-4-Quantifiers.ipynb)** - Quantificateurs
- **[Lean-5-Tactics.ipynb](Lean-5-Tactics.ipynb)** - Tactiques

**Partie 2 : Etat de l'art**
- **[Lean-6-Mathlib-Essentials.ipynb](Lean-6-Mathlib-Essentials.ipynb)** - Mathlib4
- **[Lean-7-LLM-Integration.ipynb](Lean-7-LLM-Integration.ipynb)** - Integration LLM
- **[Lean-8-Agentic-Proving.ipynb](Lean-8-Agentic-Proving.ipynb)** - Agents autonomes

---

**Navigation** : ← (debut) | [Index](Lean-1-Setup.ipynb) | [Lean-2-Dependent-Types →](Lean-2-Dependent-Types.ipynb)